# Esercizio 3

EMBL (http://www.ebi.ac.uk/cgi-bin/sva/sva.pl/) è una banca di sequenze nucleotidiche sviluppata da EMBL-EBI (European Bioinformatics Institute, European Molecular Biology Laboratory), in cui ogni sequenza nucleotidica viene memorizzata, con altre informazioni, in file di testo (*entry* EMBL) in un formato che prende il nome di formato EMBL.

**Il formato EMBL** è composto da *record* che iniziano con un codice a due lettere maisucole che specifica il contenuto del *record*. I soli *record* che non iniziano con il codice a due lettere sono quelli contenenti la sequenza nucleotidica.

Dato un file in formato EMBL, contenente la sequenza nucleotidica (sequenza di basi) di un mRNA (trascritto espresso da un gene), produrre:

- la sequenza nucleotidica in formato FASTA
- la sequenza della proteina espressa dal gene in formato FASTA

***

Parametri in input:

- nome del file in formato EMBL

***

Dove trovare le informazioni che servono per risolvere l'esercizio:

- Il record che inizia con ID
 
       ID   M10051; SV 1; linear; mRNA; STD; HUM; 4723 BP.

contiene l'identificatore univoco della sequenza (M10051) e l'organismo (HUM). Il fatto che il file si riferisca alla sequenza nucleotidica di un gene è indicato dalla presenza della parola `mRNA`.

- L'insieme dei record che iniziano con FT sono quelli che contengono le *features* della sequenza nucleotidica. In particolare tutti i record della sezione:

       FT                   /translation="MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT
       FT                   RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP
       FT                   NLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRI
       FT                   LDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTI
       FT                   [...]
       FT                   DGGSSLGFKRSYEEHIPYTHMNGGKKNGRILTLPRSNPS"
        
contengono la sequenza della proteina espressa dal gene.

- Il record che inizia con SQ:

        SQ   Sequence 4723 BP; 1068 A; 1298 C; 1311 G; 1046 T; 0 other;
        
introduce la sezione della sequenza nucleotidica che termina con il record `//` (file del file). Ogni *record* contenente la sequenza nucleotidica inizia con una serie di spazi iniziali, e contiene un *chunk* di sequenza lungo 60 basi. L'intero alla fine del *record* fornisce la lunghezza totale dei *chunks* fino a tale *record*. Ogni *chunk* in un *record* viene poi separato in *chunks* più piccoli di 10 basi.

    SQ   Sequence 4723 BP; 1068 A; 1298 C; 1311 G; 1046 T; 0 other;
         ggggggctgc gcggccgggt cggtgcgcac acgagaagga cgcgcggccc ccagcgctct        60
         tgggggccgc ctcggagcat gacccccgcg ggccagcgcc gcgcgcctga tccgaggaga       120
         ccccgcgctc ccgcagccat gggcaccggg ggccggcggg gggcggcggc cgcgccgctg       180
         ctggtggcgg tggccgcgct gctactgggc gccgcgggcc acctgtaccc cggagaggtg       240
         tgtcccggca tggatatccg gaacaacctc actaggttgc atgagctgga gaattgctct       300
         gtcatcgaag gacacttgca gatactcttg atgttcaaaa cgaggcccga agatttccga       360
         gacctcagtt tccccaaact catcatgatc actgattact tgctgctctt ccgggtctat       420
         gggctcgaga gcctgaagga cctgttcccc aacctcacgg tcatccgggg atcacgactg       480
         [...]
         tttttcgttc cccccacccg cccccagcag atggaaagaa agcacctgtt tttacaaatt      4620
         cttttttttt tttttttttt tttttttttg ctggtgtctg agcttcagta taaaagacaa      4680
         aacttcctgt ttgtggaaca aaatttcgaa agaaaaaacc aaa                        4723
    //
    
**NOTA BENE**:
- l'alfabeto degli amminoacidi è {ACDEFGHIKLMNPQRSTVWY}
- la sequenza nucleotidica riportata nell'*entry EMBL* è su alfabeto {a,c,g,t} nonostante rappresenti la sequenza primaria di un mRNA. Per ottenere la sequenza su alfabeto {a,c,g,u} basta operare una sostituzione di tutti i simboli `t` con simboli `u`.

***

Requisiti:

- nell’*header* della sequenza nucleotidica in formato FASTA devono comparire l’identificatore univoco della sequenza e l’organismo a cui si riferisce, nel seguente formato:

        >M10051-HUM
        
- la sequenza nucleotidica deve essere prodotta su alfabeto {a,c,g,u}
        
- nell’*header* della sequenza della proteina in formato FASTA devono comparire l’identificatore univoco della sequenza, l’organismo e la lunghezza della proteina, nel seguente formato:

        >M10051-HUM; len = 1427
        
- le sequenze in formato FASTA devono essere prodotte in righe di 80 caratteri

- deve essere definita una funzione `format_fasta()` che prenda come argomenti un'intestazione FASTA, una sequenza nucleotidica/proteica, e restituisca la sequenza in formato FASTA con la sequenza separata in righe di 80 caratteri.

- usare solo espressioni regolari per estrarre le informazioni

***

Variabili di output:
- `nucleotide_sequence_fasta`: sequenza nucleotidica in formato FASTA
- `protein_sequence_fasta`: sequenza della proteina in formato FASTA

***

## Soluzione

### Definizione della funzione `format_fasta()`

In [1]:
def format_fasta(header, sequence):
    return header + '\n' + '\n'.join(re.findall('\w{1,80}', sequence))

**NOTA BENE**: supporre che l'*header* in input alla funzione non abbia il simbolo *newline* `\n` alla fine ma che abbia il simbolo `>` all'inizio.

### Parametri in input

In [2]:
input_file_name = './M10051.txt'

Importazione del modulo `re` per utilizzare le espressioni regolari (*RE*).

In [3]:
import re

### Lettura del file (*entry* EMBL) in un'unica stringa `file_str`

In [4]:
with open(input_file_name,'r') as input_file:
    file_str = input_file.read()

In [5]:
file_str

### Estrazione dell'identificatore univoco e dell'organismo dell'*entry* EMBL

Estrarre dal *record* ID l'identificatore univoco e l'organismo nelle variabili `identifier` e `organism`.

    ID   M10051; SV 1; linear; mRNA; STD; HUM; 4723 BP.
    
Estrazione dell'organismo.

In [6]:
s = re.search('([\w\s]+;){5}\s+(\w+);', file_str, re.M)
organism = s.group(2)

In [7]:
organism

'HUM'

Estrazione dell'identificatore.

In [8]:
s = re.search('^ID\s+(\w+);', file_str, re.M)
identifier = s.group(1)

In [9]:
identifier

'M10051'

### Produzione della sequenza nucleotidica in formato FASTA

Estrarre nella lista `seq_row_list` i *record* della sequenza nucleotidica escludendo solo l'intero finale (mantenendo gli spazi iniziali e gli spazi prima dell'intero).

    tgggggccgc ctcggagcat gacccccgcg ggccagcgcc gcgcgcctga tccgaggaga       120

In [10]:
seq_row_list = re.findall('^\W{2}(\D+)\d+', file_str, re.M)

In [11]:
seq_row_list

['   ggggggctgc gcggccgggt cggtgcgcac acgagaagga cgcgcggccc ccagcgctct        ',
 '   tgggggccgc ctcggagcat gacccccgcg ggccagcgcc gcgcgcctga tccgaggaga       ',
 '   ccccgcgctc ccgcagccat gggcaccggg ggccggcggg gggcggcggc cgcgccgctg       ',
 '   ctggtggcgg tggccgcgct gctactgggc gccgcgggcc acctgtaccc cggagaggtg       ',
 '   tgtcccggca tggatatccg gaacaacctc actaggttgc atgagctgga gaattgctct       ',
 '   gtcatcgaag gacacttgca gatactcttg atgttcaaaa cgaggcccga agatttccga       ',
 '   gacctcagtt tccccaaact catcatgatc actgattact tgctgctctt ccgggtctat       ',
 '   gggctcgaga gcctgaagga cctgttcccc aacctcacgg tcatccgggg atcacgactg       ',
 '   ttctttaact acgcgctggt catcttcgag atggttcacc tcaaggaact cggcctctac       ',
 '   aacctgatga acatcacccg gggttctgtc cgcatcgaga agaacaatga gctctgttac       ',
 '   ttggccacta tcgactggtc ccgtatcctg gattccgtgg aggataatca catcgtgttg       ',
 '   aacaaagatg acaacgagga gtgtggagac atctgtccgg gtaccgcgaa gggcaagacc       ',
 '   aactgccccg ccaccgtcat caacgggcag t

Estrarre da `seq_row_list` la lista `seq_chunk_list` contenente i *chunks* di (al più) lunghezza 10 della sequenza nucleotidica.

**NOTA BENE**: l'elemento `seq_chunk_list[i]` è una lista annidata e contiene i sei *chunks* relativi all'i-esimo *record* di `seq_row_list`.

In [12]:
seq_chunk_list = [re.findall('\w+', row) for row in seq_row_list]

In [13]:
seq_chunk_list

[['ggggggctgc',
  'gcggccgggt',
  'cggtgcgcac',
  'acgagaagga',
  'cgcgcggccc',
  'ccagcgctct'],
 ['tgggggccgc',
  'ctcggagcat',
  'gacccccgcg',
  'ggccagcgcc',
  'gcgcgcctga',
  'tccgaggaga'],
 ['ccccgcgctc',
  'ccgcagccat',
  'gggcaccggg',
  'ggccggcggg',
  'gggcggcggc',
  'cgcgccgctg'],
 ['ctggtggcgg',
  'tggccgcgct',
  'gctactgggc',
  'gccgcgggcc',
  'acctgtaccc',
  'cggagaggtg'],
 ['tgtcccggca',
  'tggatatccg',
  'gaacaacctc',
  'actaggttgc',
  'atgagctgga',
  'gaattgctct'],
 ['gtcatcgaag',
  'gacacttgca',
  'gatactcttg',
  'atgttcaaaa',
  'cgaggcccga',
  'agatttccga'],
 ['gacctcagtt',
  'tccccaaact',
  'catcatgatc',
  'actgattact',
  'tgctgctctt',
  'ccgggtctat'],
 ['gggctcgaga',
  'gcctgaagga',
  'cctgttcccc',
  'aacctcacgg',
  'tcatccgggg',
  'atcacgactg'],
 ['ttctttaact',
  'acgcgctggt',
  'catcttcgag',
  'atggttcacc',
  'tcaaggaact',
  'cggcctctac'],
 ['aacctgatga',
  'acatcacccg',
  'gggttctgtc',
  'cgcatcgaga',
  'agaacaatga',
  'gctctgttac'],
 ['ttggccacta',
  'tcgactggtc'

Concatenare i *chunks* della lista `seq_chunk_list` per ottenere la sequenza nucleotidica nella variabile `nucleotide_sequence`.

Sostituire in `nucleotide_sequence` tutti i simboli `t` con simboli `u`.

In [16]:
nucleotide_sequence

'ggggggcugcgcggccgggucggugcgcacacgagaaggacgcgcggcccccagcgcucuugggggccgccucggagcaugacccccgcgggccagcgccgcgcgccugauccgaggagaccccgcgcucccgcagccaugggcaccgggggccggcggggggcggcggccgcgccgcugcugguggcgguggccgcgcugcuacugggcgccgcgggccaccuguaccccggagaggugugucccggcauggauauccggaacaaccucacuagguugcaugagcuggagaauugcucugucaucgaaggacacuugcagauacucuugauguucaaaacgaggcccgaagauuuccgagaccucaguuuccccaaacucaucaugaucacugauuacuugcugcucuuccgggucuaugggcucgagagccugaaggaccuguuccccaaccucacggucauccggggaucacgacuguucuuuaacuacgcgcuggucaucuucgagaugguucaccucaaggaacucggccucuacaaccugaugaacaucacccgggguucuguccgcaucgagaagaacaaugagcucuguuacuuggccacuaucgacuggucccguauccuggauuccguggaggauaaucacaucguguugaacaaagaugacaacgaggaguguggagacaucuguccggguaccgcgaagggcaagaccaacugccccgccaccgucaucaacgggcaguuugucgaacgauguuggacucauagucacugccagaaaguuugcccgaccaucuguaagucacacggcugcaccgccgaaggccucuguugccacagcgagugccugggcaacuguucucagcccgacgaccccaccaagugcguggccugccgcaacuucuaccuggacggcagguguguggagaccugcccgcccccguacuaccacuuccaggacuggcgcugugugaacuucagcuucugccag

Produrre nella variabile `nucleotide_sequence_fasta` la sequenza nucleotidica in formato FASTA con il seguente *header*:

    >M10051-HUM

In [18]:
print(nucleotide_sequence_fasta)

>M10051-HUM
ggggggcugcgcggccgggucggugcgcacacgagaaggacgcgcggcccccagcgcucuugggggccgccucggagcau
gacccccgcgggccagcgccgcgcgccugauccgaggagaccccgcgcucccgcagccaugggcaccgggggccggcggg
gggcggcggccgcgccgcugcugguggcgguggccgcgcugcuacugggcgccgcgggccaccuguaccccggagaggug
ugucccggcauggauauccggaacaaccucacuagguugcaugagcuggagaauugcucugucaucgaaggacacuugca
gauacucuugauguucaaaacgaggcccgaagauuuccgagaccucaguuuccccaaacucaucaugaucacugauuacu
ugcugcucuuccgggucuaugggcucgagagccugaaggaccuguuccccaaccucacggucauccggggaucacgacug
uucuuuaacuacgcgcuggucaucuucgagaugguucaccucaaggaacucggccucuacaaccugaugaacaucacccg
ggguucuguccgcaucgagaagaacaaugagcucuguuacuuggccacuaucgacuggucccguauccuggauuccgugg
aggauaaucacaucguguugaacaaagaugacaacgaggaguguggagacaucuguccggguaccgcgaagggcaagacc
aacugccccgccaccgucaucaacgggcaguuugucgaacgauguuggacucauagucacugccagaaaguuugcccgac
caucuguaagucacacggcugcaccgccgaaggccucuguugccacagcgagugccugggcaacuguucucagcccgacg
accccaccaagugcguggccugccgcaacuucuaccuggacggcagguguguggagaccugcccgcccccguacuaccac
uuccaggacuggcgcu

### Produzione della sequenza della proteina in formato FASTA

Estrarre nella variabile `protein_prefix` il prefisso della proteina contenuto nel *record* contenente la parola `/translation`:

    FT                   /translation="MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT

In [21]:
protein_prefix

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT'

Estrarre nella lista `protein_row_list` gli altri *record* (compreso l'ultimo) che contengono la sequenza della proteina.

    FT                   RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP
    
Attenzione all'ultimo:

    FT                   DGGSSLGFKRSYEEHIPYTHMNGGKKNGRILTLPRSNPS"


In [24]:
protein_row_list

['RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP',
 'NLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRI',
 'LDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTI',
 'CKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCV',
 'NFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCH',
 'LLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRR',
 'SYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYN',
 'PKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWE',
 'PYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHP',
 'GWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSS',
 'SQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDS',
 'QKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAE',
 'DPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGL',
 'RHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMW',
 'QEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSL',
 'AGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLF

Aggiungere alla lista il prefisso trovato prima e concatenare nella variabile `protein_sequence` tutti i blocchi della lista `protein_row_list` per ottenere la sequenza della proteina.

In [28]:
protein_sequence

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTMGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSLAGNGSWTEPTYFYVTDYLDVPSNI

Produrre nella variabile `protein_sequence_fasta` la sequenza della proteina in formato FASTA con il seguente *header*:

    >M10051-HUM; len = 1427

In [30]:
print(protein_sequence_fasta)

>M10051-HUM; len = 1427
MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTMGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVC
PGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLF
FNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNHIVLNKDDNEECGDICPGTAKGKTN
CPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHF
QDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQ
ELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNL
RQLWDWSKHNLTTTQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLR
WEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKT
LVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCL
KGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDP
RPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCS
VAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRG
LSPG